In [1]:
import numpy as np
import pandas as pd

In [3]:
print("hello world")

hello world


In [ ]:
print("done this after the commit")
### asaas

In [5]:
df = pd.read_csv('._reviews_202106.csv', encoding='unicode_escape')
print(df)

  Unnamed: 0
0          
1  cbÓéS´
